Steps:


1. Create data model
    - attributes
    - flags
    - groups

2. Process data model
    - index nodes 
    - infer nodes 
    - detect networks
    
**3. Explore networks**

4. Generate AI reports

# 3. Explore networks

In [ ]:
%pip install ucimlrepo
import sys
from ucimlrepo import fetch_ucirepo
import polars as pl

sys.path.append("../..")

computer_hardware  = fetch_ucirepo(id=29)


input_dataframe = pl.DataFrame(computer_hardware.data.original)
pl.Config.set_fmt_str_lengths(200)
input_dataframe.head()

entity_id_column = "VendorName"
columns_to_link = ["ModelName", "MYCT", "MMIN", "MMAX", "CACH", "CHMIN", "CHMAX", "PRP", "ERP"]
pl.Config.set_fmt_str_lengths(200)


## Detect networks

In [ ]:
# get native-country and randomly put flags for them
names = input_dataframe["VendorName"].unique().to_list()

# random numbers between 0 and 5 in a dataframe with id, flags and value
n = 500
df = pl.DataFrame(
    {
        "id": pl.Series("id", list(range(n))),
        "flags": pl.Series("flags", [names[i % len(names)] for i in range(n)]),
        "value": pl.Series("value", [i % 5 for i in range(n)]),
    }
)

## Network data by Entity 

In [ ]:
from toolkit.detect_entity_networks.main import build_simple_network_data


entity_records, nodes, edges = build_simple_network_data(input_dataframe, entity_id_column, columns_to_link)

In [ ]:
entity_df = pl.DataFrame(
                    entity_records,
                    schema=[
                        "Entity ID",
                        "Entity Flags",
                        "Network ID",
                        "Network Entities",
                        "Network Flags",
                        "Flagged",
                        "Flags/Entity",
                        "Flagged/Unflagged",
                    ],
                ).sort("Flagged/Unflagged", descending=True)

entity_df.head()

In [ ]:
nodes_df = pl.DataFrame(
                    nodes,
                    schema=[
                        "title",
                        "id",
                    ],
                )
nodes_df.head()